In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import PIL.Image as Image
import cv2
import copy
import time

In [ ]:
NUM_CLASS=3
CROP_SIZE=160
NUM_FRAMES_PER_CLIP=16
BATCH_SIZE=12
RGB_CHANNEL=3
IS_TRAIN=True
BLOCK_EXPANSION=4
IS_DA=True  #True if you using data augmentation

In [ ]:
def get_conv_weight(name,kshape,wd=0.0005):
    with tf.device('/cpu:0'):
        var=tf.get_variable(name,shape=kshape,initializer=tf.contrib.layers.xavier_initializer())
    if wd!=0:
        weight_decay = tf.nn.l2_loss(var)*wd
        tf.add_to_collection('weightdecay_losses', weight_decay)
    return var

In [ ]:
def convS(name,l_input,in_channels,out_channels):
    return tf.nn.bias_add(tf.nn.conv3d(l_input,get_conv_weight(name=name,
                                                               kshape=[1,3,3,in_channels,out_channels]),
                                                               strides=[1,1,1,1,1],padding='SAME'),
                                              get_conv_weight(name+'_bias',[out_channels],0))
def convT(name,l_input,in_channels,out_channels):
    return tf.nn.bias_add(tf.nn.conv3d(l_input,get_conv_weight(name=name,
                                                               kshape=[3,1,1,in_channels,out_channels]),
                                                               strides=[1,1,1,1,1],padding='SAME'),
                                              get_conv_weight(name+'_bias',[out_channels],0))

In [ ]:
class Bottleneck():
    def __init__(self,l_input,inplanes,planes,stride=1,downsample='',n_s=0,depth_3d=47):
        
        self.X_input=l_input
        self.downsample=downsample
        self.planes=planes
        self.inplanes=inplanes
        self.depth_3d=depth_3d
        self.ST_struc=('A','B','C')
        self.len_ST=len(self.ST_struc)
        self.id=n_s
        self.n_s=n_s
        self.ST=list(self.ST_struc)[self.id % self.len_ST]
        self.stride_p=[1,1,1,1,1]
       
        if self.downsample!='':
            self.stride_p=[1,1,2,2,1]
        if n_s<self.depth_3d:
            if n_s==0:
                self.stride_p=[1,1,1,1,1]
        else:
            if n_s==self.depth_3d:
                self.stride_p=[1,2,2,2,1]
            else:
                self.stride_p=[1,1,1,1,1]
    #P3D has three types of bottleneck sub-structions.
    def ST_A(self,name,x):
        x=convS(name+'_S',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        x=convT(name+'_T',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        return x
    
    def ST_B(self,name,x):
        tmp_x=convS(name+'_S',x,self.planes,self.planes)
        tmp_x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
        tmp_x=tf.nn.relu(tmp_x)
        x=convT(name+'_T',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        return x+tmp_x
    
    def ST_C(self,name,x):
        x=convS(name+'_S',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        tmp_x=convT(name+'_T',x,self.planes,self.planes)
        tmp_x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
        tmp_x=tf.nn.relu(tmp_x)
        return x+tmp_x
    
    def infer(self):
        residual=self.X_input
        if self.n_s<self.depth_3d:
            out=tf.nn.conv3d(self.X_input,get_conv_weight('conv3_{}_1'.format(self.id),[1,1,1,self.inplanes,self.planes]),
                             strides=self.stride_p,padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
            
        else:
            param=self.stride_p
            param.pop(1)
            out=tf.nn.conv2d(self.X_input,get_conv_weight('conv2_{}_1'.format(self.id),[1,1,self.inplanes,self.planes]),
                             strides=param,padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
    
        out=tf.nn.relu(out)    
        if self.id<self.depth_3d:
            if self.ST=='A':
                out=self.ST_A('STA_{}_2'.format(self.id),out)
            elif self.ST=='B':
                out=self.ST_B('STB_{}_2'.format(self.id),out)
            elif self.ST=='C':
                out=self.ST_C('STC_{}_2'.format(self.id),out)
        else:
            out=tf.nn.conv2d(out,get_conv_weight('conv2_{}_2'.format(self.id),[3,3,self.planes,self.planes]),
                                  strides=[1,1,1,1],padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
            out=tf.nn.relu(out)

        if self.n_s<self.depth_3d:
            out=tf.nn.conv3d(out,get_conv_weight('conv3_{}_3'.format(self.id),[1,1,1,self.planes,self.planes*BLOCK_EXPANSION]),
                             strides=[1,1,1,1,1],padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
        else:
            out=tf.nn.conv2d(out,get_conv_weight('conv2_{}_3'.format(self.id),[1,1,self.planes,self.planes*BLOCK_EXPANSION]),
                             strides=[1,1,1,1],padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
           
        if len(self.downsample)==1:
            residual=tf.nn.conv2d(residual,get_conv_weight('dw2d_{}'.format(self.id),[1,1,self.inplanes,self.planes*BLOCK_EXPANSION]),
                                  strides=[1,2,2,1],padding='SAME')
            residual=tf.layers.batch_normalization(residual,training=IS_TRAIN)
        elif len(self.downsample)==2:
            residual=tf.nn.conv3d(residual,get_conv_weight('dw3d_{}'.format(self.id),[1,1,1,self.inplanes,self.planes*BLOCK_EXPANSION]),
                                  strides=self.downsample[1],padding='SAME')
            residual=tf.layers.batch_normalization(residual,training=IS_TRAIN)
        out+=residual
        out=tf.nn.relu(out)
        
        return out

In [ ]:
class make_block():
    def __init__(self,_X,planes,num,inplanes,cnt,depth_3d=47,stride=1):
        self.input=_X
        self.planes=planes
        self.inplanes=inplanes
        self.num=num
        self.cnt=cnt
        self.depth_3d=depth_3d
        self.stride=stride
        if self.cnt<depth_3d:
            if self.cnt==0:
                stride_p=[1,1,1,1,1]
            else:
                stride_p=[1,1,2,2,1]
            if stride!=1 or inplanes!=planes*BLOCK_EXPANSION:
                self.downsample=['3d',stride_p]
        else:
            if stride!=1 or inplanes!=planes*BLOCK_EXPANSION:
                self.downsample=['2d']
    def infer(self):
        x=Bottleneck(self.input,self.inplanes,self.planes,self.stride,self.downsample,n_s=self.cnt,depth_3d=self.depth_3d).infer()
        self.cnt+=1
        self.inplanes=BLOCK_EXPANSION*self.planes
        for i in range(1,self.num):
            x=Bottleneck(x,self.inplanes,self.planes,n_s=self.cnt,depth_3d=self.depth_3d).infer()
            self.cnt+=1
        return x

In [ ]:
def inference_p3d(_X,_dropout,BATCH_SIZE):
    cnt=0
    conv1_custom=tf.nn.conv3d(_X,get_conv_weight('firstconv1',[1,7,7,RGB_CHANNEL,64]),strides=[1,1,2,2,1],padding='SAME')
    conv1_custom_bn=tf.layers.batch_normalization(conv1_custom,training=IS_TRAIN)
    conv1_custom_bn_relu=tf.nn.relu(conv1_custom_bn)
    x=tf.nn.max_pool3d(conv1_custom_bn_relu,[1,2,3,3,1],strides=[1,2,2,2,1],padding='SAME')
    b1=make_block(x,64,3,64,cnt)
    x=b1.infer()
    cnt=b1.cnt
   
    x=tf.nn.max_pool3d(x,[1,2,1,1,1],strides=[1,2,1,1,1],padding='SAME')
    
    b2=make_block(x,128,8,256,cnt,stride=2)
    x=b2.infer()
    cnt=b2.cnt
    x=tf.nn.max_pool3d(x,[1,2,1,1,1],strides=[1,2,1,1,1],padding='SAME')
    
    b3=make_block(x,256,36,512,cnt,stride=2)
    x=b3.infer()
    cnt=b3.cnt
    x=tf.nn.max_pool3d(x,[1,2,1,1,1],strides=[1,2,1,1,1],padding='SAME')
    
    shape=x.shape.as_list()
    x=tf.reshape(x,shape=[-1,shape[2],shape[3],shape[4]])
    
    x=make_block(x,512,3,1024,cnt,stride=2).infer()
    
    #Caution:make sure avgpool on the input which has the same shape as kernelsize has been setted padding='VALID'
    x=tf.nn.avg_pool(x,[1,5,5,1],strides=[1,1,1,1],padding='VALID')
    
    x=tf.reshape(x,shape=[-1,2048])
    if(IS_TRAIN):
        x=tf.nn.dropout(x,keep_prob=0.5)
    else:
        x=tf.nn.dropout(x,keep_prob=1)
    
    x=tf.layers.dense(x,NUM_CLASS)
    
    return x

In [ ]:
def compute_loss(name_scope,logit,labels):
    cross_entropy_mean=tf.reduce_mean(
                    tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=logit))
    tf.summary.scalar(name_scope+'_cross_entropy',
                     cross_entropy_mean
                     )
    weight_decay_loss=tf.get_collection('weightdecay_losses')
    tf.summary.scalar(name_scope+'_weight_decay_loss',tf.reduce_mean(weight_decay_loss))
    total_loss=cross_entropy_mean+weight_decay_loss
    tf.summary.scalar(name_scope+'_total_loss',tf.reduce_mean(total_loss))
    return total_loss

In [ ]:
def compute_accuracy(logit,labels):
    correct=tf.equal(tf.argmax(logit,1),labels)
    acc=tf.reduce_mean(tf.cast(correct,tf.float32))
    return acc

In [ ]:
class DataAugmenter():
    def __init__(self,isFlip=True,isShift=True,isScale=True,isBrightness=True,isHistogram_eq=True):
        #flipcode depends on your own dataset.
        self.tp=0
        self.angle_val=np.random.uniform(6,10)
        self.angle=np.random.choice([-self.angle_val,self.angle_val])
        self.scale=np.random.uniform(1.0,1.1)
        self.random_br=np.random.uniform(0.5,2.0)
        self.x_shift=None
        self.y_shift=None
    def Flip(self):
        if self.tp==0:
            self.input=cv2.flip(self.input,1)
        else:
            self.input=cv2.flip(self.input,0)
    def Shift(self):
        M=np.float32([[1,0,self.x_shift],[0,1,self.y_shift]])  
        self.input=cv2.warpAffine(self.input,M,(self.cols,self.rows))
        
    def Scale(self):
        M=cv2.getRotationMatrix2D((self.cols/2,self.rows/2),0,self.scale)
        self.input=cv2.warpAffine(self.input, M, (self.cols, self.rows))
        
    def Brightness(self):
        hsv=cv2.cvtColor(self.input,cv2.COLOR_RGB2HSV)
        mask=hsv[:,:,2] * self.random_br >255
        v_channel=np.where(mask,255,hsv[:,:,2] * self.random_br)
        hsv[:,:,2]=v_channel
        self.input=cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
        
    def Histogram(self):
        tpimg=cv2.cvtColor(self.input,cv2.COLOR_RGB2HSV)
        tpimg[:,:,2] = cv2.equalizeHist(tpimg[:,:,2])
        self.input=cv2.cvtColor(tpimg,cv2.COLOR_HSV2RGB)
    
    def Rotate(self): 
        M=cv2.getRotationMatrix2D((self.cols/2,self.rows/2),self.angle,1)
        self.input=cv2.warpAffine(self.input, M, (self.cols, self.rows)) 
        
    def show(self):
        self.input=Image.fromarray(self.input)
        self.input.show()
        self.input=np.array(self.input)
    
    def Apply(self,_Clip):
        res=[]
        #set the possibility of all measures of DA:
        # 50 % possibility for Flip , Rotate , Scale , Brightness changing , Histogram-equal ops.
        pro_flip=random.choice([0,1])
        pro_rotate=random.choice([0,1])
        #pro_shift=random.choice([0,1])
        pro_scale=random.choice([0,1])
        pro_bri=random.choice([0,1])
        pro_his=random.choice([0,1])
        # DO NOT USE SHIFT
        pro_shift=0 
        #reset new property of DA
        self.__init__()
        for pic in _Clip:
            self.input=pic
            self.rows,self.cols,_=pic.shape
            if pro_his:
                self.Histogram()   
                
            if pro_bri:
                self.Brightness()
                
            if pro_flip:  
                self.Flip()
            if pro_rotate:
                self.Rotate()
            if pro_shift:
                if self.x_shift==None:     
                    x=np.random.randint(self.cols/20,self.cols/16)
                    y=np.random.randint(self.rows/20,self.rows/16)
                    x_shift=random.choice([-x,x])
                    y_shift=random.choice([-y,y])
                    self.x_shift=x_shift
                    self.y_shift=y_shift
                self.Shift()
            if pro_scale:
                self.Scale()
            res.append(self.input)
        return res 

In [ ]:
#Utils function for extracting frames of each video to compose a clip.
DA=DataAugmenter()

def get_frames_data(filename,num_frames_per_clip=16,is_da=False):
    
    ret_arr=[]
    s_index=0
    for parent, dirnames,filenames in os.walk(filename):
        if(len(filenames)<num_frames_per_clip):
            print("Get invaild data!")
            return [],s_index
        filenames=sorted(filenames)
        s_index=random.randint(0,len(filenames)-num_frames_per_clip)
        for i in range(s_index,s_index+num_frames_per_clip):
            image_name=str(filename)+'/'+str(filenames[i])
            img=Image.open(image_name)
            img_data=np.array(img)
            ret_arr.append(img_data)
    if is_da:
        return DA.Apply(ret_arr),s_index
    else:
        return ret_arr,s_index

In [ ]:
class DataGenerator:
    def __init__(self,filename,batch_size,num_frames_per_clip,shuffle=True,crop_size=160,is_da=IS_DA):
        
        self.index=0
        self.lines=open(filename,'r')
        self.lines=list(self.lines)
        self.len=len(self.lines)
        self.batch_size=batch_size
        self.num_frames_per_clip=num_frames_per_clip
        self.indexlist=[]
        self.crop_size=crop_size
        self.is_da=is_da
        if shuffle:
            self.video_indices=range(len(self.lines))
            random.seed(time.time())
            random.shuffle(self.video_indices)
        else:
            self.video_indices=range(0,len(self.lines))
    def next_batch(self):
        data=[]
        labels=[]
        crop_size=self.crop_size
        self.indexlist=[]
        
        if self.index + self.batch_size > self.len:
            self.index=0
        for index in self.video_indices[self.index:self.index+self.batch_size]:
            self.indexlist.append(index)
            line=self.lines[index].strip('\n').split()
            dirname=line[0]
            label=line[1]
    
            tmp_data,_=get_frames_data(dirname,self.num_frames_per_clip,self.is_da)
            img_datas=[]
            if(len(tmp_data)!=0):
#                 first=True    
                for j in xrange(len(tmp_data)):
                    
                    img=Image.fromarray(tmp_data[j].astype(np.uint8))  
                        
                    if(img.width>img.height):
                        scale=float(crop_size)/float(img.height)
                        img=np.array(cv2.resize(np.array(img),(int(img.width*scale+1),crop_size))).astype(np.float32)      
                    else:                    
                        scale=float(crop_size)/float(img.width)
                        img=np.array(cv2.resize(np.array(img),(crop_size,int(img.height*scale+1)))).astype(np.float32)
                    crop_x=int((img.shape[0]-crop_size)/2)
                    crop_y=int((img.shape[1]-crop_size)/2)
                    img=img[crop_x:crop_x+crop_size,crop_y:crop_y+crop_size,:] #-np_mean[j] 
                    img_datas.append(img)
#                     if first:
#                         t_img=img.astype(np.uint8)
#                         Image.fromarray(t_img).show()
#                         first=False
                data.append(img_datas)
                labels.append(int(label))
               
        self.index+=self.batch_size          
        return np.array(data).astype(np.float32),np.array(labels).astype(np.int64),self.indexlist

In [ ]:
#Create dataloader to fetch each batch data.
dataloader=DataGenerator(filename='train.list',
                                batch_size=BATCH_SIZE,
                                num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                                shuffle=True,is_da=False)

In [ ]:
MOVING_AVERAGE_DECAY=0.9
MODEL_PATH=''
USE_PRETRAIN=False
MAX_STEPS=36000

In [ ]:
# #Test Result for Singe Batch :
# IS_TRAIN=False

# input_placeholder=tf.placeholder(tf.float32,shape=(BATCH_SIZE,NUM_FRAMES_PER_CLIP,CROP_SIZE,CROP_SIZE,RGB_CHANNEL))
# label_placeholder=tf.placeholder(tf.int64,shape=(BATCH_SIZE))
# logit=inference_p3d(input_placeholder,1,BATCH_SIZE)
# acc=compute_accuracy(logit,label_placeholder)
# loss=compute_loss('default_loss',logit,label_placeholder)
# saver=tf.train.Saver(tf.global_variables())
# init=tf.global_variables_initializer()
# sess=tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
# if USE_PRETRAIN:
#     saver.restore(sess,MODEL_PATH)
#     print('checkpoint reloaded.')
# else:
#     print('train from sratch.')
# train_images,train_labels,_=dataloader.next_batch()
# curacc,curloss,curlogit=sess.run([acc,loss,logit],feed_dict={
#                         input_placeholder:train_images,
#                         label_placeholder:train_labels})
# print(curacc,np.mean(curloss))
# print(curlogit)
# print(train_labels)

In [ ]:
# Make sure IS_TRAIN==True before traininig.
IS_TRAIN

In [ ]:
#Cell for Training.

with tf.Graph().as_default():
    global_step=tf.get_variable('global_step',[],initializer=tf.constant_initializer(0),trainable=False)
    
    input_placeholder=tf.placeholder(tf.float32,shape=(BATCH_SIZE,NUM_FRAMES_PER_CLIP,CROP_SIZE,CROP_SIZE,RGB_CHANNEL))
    label_placeholder=tf.placeholder(tf.int64,shape=(BATCH_SIZE))
    
    #set dropout to 0.5
    logit=inference_p3d(input_placeholder,0.5,BATCH_SIZE)
    acc=compute_accuracy(logit,label_placeholder)
    tf.summary.scalar('accuracy',acc)
    loss=compute_loss('default_loss',logit,label_placeholder)
    
    
    varlist1=[]
    varlist2=[]
    for param in tf.trainable_variables():
        if param.name!='dense/bias:0' and param.name!='dense/kernel:0':
            varlist1.append(param)
        else:
            varlist2.append(param)
    
    learning_rate_stable = tf.train.exponential_decay(0.0005,
                                               global_step,decay_steps=2100,decay_rate=0.6,staircase=True)
    learning_rate_finetune = tf.train.exponential_decay(0.0005,
                                               global_step,decay_steps=2100,decay_rate=0.6,staircase=True)
    
    opt_stable=tf.train.AdamOptimizer(learning_rate_stable)
    opt_finetuning=tf.train.AdamOptimizer(learning_rate_finetune)
    
    #when using BN,this dependecy must be built.
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) 
    optim_op1=opt_stable.minimize(loss,var_list=varlist1)
    optim_op2=opt_finetuning.minimize(loss,var_list=varlist2,global_step=global_step)
    
    with tf.control_dependencies(update_ops):
        optim_op_group=tf.group(optim_op1,optim_op2)
        
    
    
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,num_updates=global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    
    train_op=tf.group(optim_op_group,variable_averages_op)
    
    #when using BN,only store trainable parameters is not enough,cause MEAN and VARIANCE for BN is not
    #trainable but necessary for test stage.
    saver=tf.train.Saver(tf.global_variables())
    init=tf.global_variables_initializer()
    sess=tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    if USE_PRETRAIN:
        saver.restore(sess,MODEL_PATH)
        print('checkpoint reloaded.')
    else:
        print('train from sratch.')
    merged=tf.summary.merge_all()
    train_writer=tf.summary.FileWriter('./visual_logs/train',sess.graph)
    test_writer=tf.summary.FileWriter('./visual_logs/test',sess.graph)
    duration=0
    print('Start training.')
    for step in xrange(1,MAX_STEPS):
        sess.graph.finalize()
        start_time=time.time()
        train_images,train_labels,_=dataloader.next_batch()
        sess.run(train_op,feed_dict={
                        input_placeholder:train_images,
                        label_placeholder:train_labels})
        duration+=time.time()-start_time
        
        
        if step!=0 and step % 10==0:
            curacc,curloss=sess.run([acc,loss],feed_dict={
                        input_placeholder:train_images,
                        label_placeholder:train_labels})
            print('Step %d: %.2f sec -->loss : %.4f =====acc : %.2f' % (step, duration,np.mean(curloss),curacc))
            duration=0
        if step!=0 and step % 50==0:
            mer=sess.run(merged,feed_dict={
                        input_placeholder:train_images,
                        label_placeholder:train_labels})
            train_writer.add_summary(mer, step)
        if step >7000 and step % 800==0 or (step+1)==MAX_STEPS:
            saver.save(sess,'./TFCHKP_{}'.format(step),global_step=step)
        
    print('done')   
            
        
    

In [ ]:
#Cell for Testing
MOVING_AVERAGE_DECAY=0.99
tf.reset_default_graph()
#when testing ,make sure IS_TRAIN==False,or you will get bad result for testing.
IS_TRAIN=False
final_acc=0
IS_DA=False
testloader=DataGenerator(filename='testrr.list',
                                batch_size=BATCH_SIZE,
                                num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                                shuffle=False,is_da=IS_DA)

c=0
with tf.Graph().as_default():
    
    global_step=tf.get_variable('global_step',[],initializer=tf.constant_initializer(0),trainable=False)
    input_placeholder=tf.placeholder(tf.float32,shape=(BATCH_SIZE,NUM_FRAMES_PER_CLIP,CROP_SIZE,CROP_SIZE,RGB_CHANNEL))
    label_placeholder=tf.placeholder(tf.int64,shape=(BATCH_SIZE))
    
    #when testing,make sure dropout=1.0(keep_prob)
    logit=inference_p3d(input_placeholder,1,BATCH_SIZE)
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,num_updates=global_step)
    variable_averages_op=variable_averages.apply(tf.trainable_variables())
    acc=compute_accuracy(logit,label_placeholder)
    init=tf.global_variables_initializer()
    variable_avg_restore=variable_averages.variables_to_restore()
    
    avglist=[]

    saver=tf.train.Saver(tf.global_variables())
    # You can also restore the moving_average parameters ,like this:
    # saver=tf.train.Saver(variable_avg_restore)
    
    sess=tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    #restore your checkpoint file
    saver.restore(sess,'da_tfckp_it_33600-33600')

    for step in range(testloader.len/BATCH_SIZE):
        image,label,_=testloader.next_batch()
        accuracy=sess.run(acc,feed_dict={input_placeholder:image,
                                    label_placeholder:label})
        print('->',accuracy)
        final_acc+=accuracy
        c+=1
    print(final_acc/c)

In [ ]:
IS_DA

In [ ]:
#Cell for Testing :Using the same checkpoint file,you can continue testing without resotring checkpoint again.
testloader=DataGenerator(filename='res.list',
                                batch_size=BATCH_SIZE,
                                num_frames_per_clip=NUM_FRAMES_PER_CLIP,
                                shuffle=False,is_da=IS_DA)
final_acc=0
c=0
with tf.Graph().as_default():
    for step in range(testloader.len/BATCH_SIZE):
        image,label,_=testloader.next_batch()
        accuracy=sess.run(acc,feed_dict={input_placeholder:image,
                                    label_placeholder:label})
        print('->',accuracy)
        final_acc+=accuracy
        c+=1
    print(final_acc/c)